In [1]:
#IMPORT DELLE LIBRERIE
import pandas as pd
import numpy as np
import plotly.express as px

In [9]:
#IMPORT DEI DATI
df= pd.read_csv('https://github.com/LilloBarilotto/LabInternet2022/blob/main/Report2_Test_Ping_Plots/Data_txt/10Mbitdiretto.txt?raw=true',delimiter=' ')
df

,Size,RttMin,RttAvg,RttMax
0,16,0.589,0.666,0.692
1,116,1.062,1.117,1.146
2,216,1.587,1.625,1.662
3,316,2.028,2.076,2.106
4,416,2.560,2.590,2.604
...,...,...,...,...
126,9600,21.308,21.320,21.360
127,9700,21.475,21.484,21.508
128,9800,21.631,21.638,21.679
129,9900,21.793,21.805,21.822


In [3]:
#Definizioni per RTT

maxSize = 1472 #Max Size deriva da 1500=MTU - 20 byte IPHeader - 8byte ICMPHeader
L2size  = 1538 #total size with max payload =1500
headers = 58  #ICMP + IP + Ethernet

def getLenght(s):
    mod = (s%maxSize)
    a = (s // maxSize) * L2size
    if (mod != 0):
        a = a + mod + getPadding(mod) + headers
    return a

def getPadding(x):
    if (x < 18):
        return 18-x
    return 0

def getRttTheory(d, C):
    return 2*d*8*(10**3)/(C)

def getRttTheory_Switch(d, C):
    if(d<=1538):
        return 2*getRttTheory(d,C)
    return getRttTheory(d,C) + getRttTheory(1538,C) 

def getRttTheory_TwoStoreForward(d, C):
    if(d<=1538):
        return 4* getRttTheory(d ,C)
    return getRttTheory(d,C) + 2*getRttTheory(1538, C)
    
getLenght  = np.vectorize(getLenght)
getPadding = np.vectorize(getPadding)
getRttTheory_Switch= np.vectorize(getRttTheory_Switch)
getRttTheory_TwoStoreForward= np.vectorize(getRttTheory_TwoStoreForward)

In [13]:
#CALCOLO VALORI EFFETTIVI

#Togli il commento a una delle due per settare la capacità
#Cap=100*10**6
Cap=10*10**6

lenght = getLenght(df['Size'])
RttTheory = getRttTheory(lenght, Cap)
RttTheorySwitch = getRttTheory_Switch(lenght, Cap)
RttTheory_TwoStoreForward = getRttTheory_TwoStoreForward(lenght, Cap)

In [14]:
#Stampa RTT

fig = px.line(df, x=lenght, y=['RttMin', 'RttAvg', 'RttMax', RttTheory, RttTheorySwitch, RttTheory_TwoStoreForward],
                markers=True ,labels=dict(x="Data [bytes]", value="RTT [ms]", variable="Tipologia di RTT"),
                width=1400, height= 900)

newnamesFig = {'RttMin':'RttMin', 'RttAvg':'RttAvg', 'RttMax':'RttMax', 'wide_variable_3': 'RttTheory','wide_variable_4': 'RttTheorySwitch(Store&Forward)', 'wide_variable_5':'RttTheory_TwoStoreForward'}
fig.for_each_trace(lambda t: t.update(name = newnamesFig[t.name], legendgroup = newnamesFig[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnamesFig[t.name])
                                     )
                  )

fig.update_layout(legend=dict(  yanchor="top", y=0.99, xanchor="left", x=0.01 ))
fig.show()

In [6]:
#Definizioni per la capacità
def getCapacity_sec(d, rtt):
    x=  2*8*d/rtt/1000
    return x

def getCapacity_sec_TheorySwitch(d, rtt):
    if(d<=1538):
        return 2*getCapacity_sec(d, rtt)
    if(Cap==10*10**6):                      # 2.4 nel caso di 10Mbit/s, altrimenti 0.24 per 100Mbit/s
        return 2*8*d/(rtt -2.4)/1000
    return 2*8*d/(rtt -0.24)/1000

getCapacity_sec_TheorySwitch= np.vectorize(getCapacity_sec_TheorySwitch)

In [7]:
#Calcolo delle capacità
Speed= getCapacity_sec(lenght, df['RttMin'])
SpeedTheory = getCapacity_sec(lenght, RttTheory)

SpeedTheorySwitch = getCapacity_sec_TheorySwitch(lenght, RttTheorySwitch)
SpeedSwitch= getCapacity_sec_TheorySwitch(lenght, df['RttMin'])

In [8]:
#STAMPA CAPACITA'

fig2 = px.line(df, x=lenght, y=[Speed, SpeedTheory, SpeedSwitch, SpeedTheorySwitch],
            labels=dict(x="Data [bytes]", value="Capacity [Mbit/s]", variable="Types of Capacity"))

newnamesFig2 = {'wide_variable_0':'Capacity(RttMin)', 'wide_variable_1':'Capacity(RttTheory)', 'wide_variable_2':'Capacity(RttMinSwitch)', 'wide_variable_3':'Capacity(RttTheorySwitch)'}
fig2.for_each_trace(lambda t: t.update(name = newnamesFig2[t.name],
                                      legendgroup = newnamesFig2[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnamesFig2[t.name])
                                     )
                  )

fig2.update_layout(legend=dict( yanchor="top", y=0.90, xanchor="left", x=0.01))
fig2.show()